In [1]:
import pandas as pd
from pymongo import MongoClient
from bson.son import SON
from bson.code import Code
import re
import pprint
import bson

client = MongoClient('localhost',27017)
db = client.recommandation_db
print(db.list_collection_names())

['precision_metric_lambda_minus_zeroPointSeven', 'recommendation_lambda_plus_zeroPointFour', 'precision_metric_lambda_minus_zeroPointOne', 'precision_metric_lambda_minus_zeroPointThree', 'recommendation_lambda_plus_zeroPointTwo', 'recommendation_lambda_minus_zeroPointTwo', 'train_set', 'recommendation_lambda_minus_onePointOne', 'train_user_movies', 'precision_metric_lambda_minus_One', 'precision_metric_lambda_plus_zeroPointThree', 'precision_metric_lambda_minus_onePointFive', 'recommendation_lambda_minus_zeroPointSix', 'precision_metric_lambda_minus_onePointOne', 'recommendation_lambda_minus_zeroPointOne', 'precision_metric_lambda_minus_zeroPointEight', 'precision_metric_lambda_minus_onePointTwo', 'recommendation_lambda_plus_zeroPointOne', 'recommendation_lambda_minus_zeroPointFour', 'recommendation_scores', 'recommendation_lambda_minus_zeroPointNine', 'recommendation_lambda_minus_onePointFive', 'precision_metric_lambda_minus_onePointThree', 'precision_metric_lambda_minus_onePointFour'

In [2]:
def precision_metric_lambda_minus_zeroPointFive(userId,L):
    cur=db.prove_set.aggregate([
        {'$match':{'userId':userId}},
        {'$out':'temporary'}
    ])
#     -------------------------------------------------------------------------------
    cur=db.recommendation_lambda_minus_zeroPointFive.aggregate([
        {'$match':{'user_I':userId}},
        {
            '$lookup':{
                'from':'temporary',
                'localField':'movieId',
                'foreignField':'movieId',
                'as':'recommendation'
            }
        },
        {'$unwind':'$recommendation'},
        {'$count':'P_50'},
        {
            '$addFields':{
                'user_I':userId
            }
        },
        {
            '$project':{
                'P_50':{'$divide':['$P_50',L]},
                'user_I':1
            }
        },
        {'$merge':'precision_metric_lambda_minus_zeroPointFive'}
    ])
#     -------------------------------------------------------------------------------
    db.temporary.drop()


In [5]:
cur=db.train_user_movies.aggregate([
    {'$sort':{
        '_id':1
    }},
    {
        '$project':{
            '_id':1
        }
    }
])
user_list=list(cur)
L=50


In [6]:
for i in range(1000,2000):
    precision_metric_lambda_minus_zeroPointFive(user_list[i]['_id'],L)

In [7]:
cur=db.precision_metric_lambda_minus_zeroPointFive.aggregate([
    {
        '$group':{
            '_id':'',
            'sum_P_50':{'$sum':'$P_50'}
        }
    }
])

In [8]:
for i in cur:
    print(i)

{'_id': '', 'sum_P_50': 63.02}


In [9]:
63.02/2000

0.03151